In [ ]:
#   LOAD ALL SAVED MODELS (CORRECT ARCHITECTURES) + TRAIN ACC

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os

# MOUNT DRIVE

from google.colab import drive
drive.mount('/content/drive')

MODELS_DIR = "/content/drive/MyDrive/NN_CIFAR10_Project"
model_files = [f for f in os.listdir(MODELS_DIR) if f.endswith(".pth")]

print("Found models:")
for f in model_files:
    print("  •", f)

# CIFAR-10 TRAINSET (same aug + normalization)

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform_train
)
trainloader = DataLoader(trainset, batch_size=128, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("\nDevice:", device)

# ARCHITECTURES

class BestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Flatten(),
            nn.Linear(3072, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 10)
        )
    def forward(self, x): return self.network(x)

class BestCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)

        self.conv5 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.conv6 = nn.Conv2d(256, 256, 3, padding=1)
        self.bn6 = nn.BatchNorm2d(256)

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)
        x = F.dropout2d(x, 0.1, training=self.training)

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, 2)
        x = F.dropout2d(x, 0.2, training=self.training)

        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = F.max_pool2d(x, 2)
        x = F.dropout2d(x, 0.3, training=self.training)

        return self.classifier(x)

class ConfigurableMLP(nn.Module):
    def __init__(self, hidden_sizes):
        super().__init__()
        layers = [nn.Flatten()]
        in_size = 3072

        for h in hidden_sizes:
            layers += [
                nn.Linear(in_size, h),
                nn.BatchNorm1d(h),
                nn.ReLU(),
                nn.Dropout(0.3)
            ]
            in_size = h

        layers.append(nn.Linear(in_size, 10))
        self.network = nn.Sequential(*layers)

    def forward(self, x): return self.network(x)

# MAP FILENAMES → CORRECT ARCHITECTURE

def build_model_for_file(filename):
    if "CNN" in filename:
        return BestCNN()
    if filename.startswith("MLP-Best"):
        return BestMLP()
    if "MLP-Small-256" in filename:
        return ConfigurableMLP([256])
    if "MLP-Large-1024-512" in filename:
        return ConfigurableMLP([1024, 512])
    if "MLP-512-LR001" in filename or "MLP-512-LR01" in filename:
        return ConfigurableMLP([512])
    raise ValueError(f"Unknown model file: {filename}")

# TRAIN ACCURACY FUNCTION

def compute_train_accuracy(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return 100 * correct / total

# LOAD MODELS + COMPUTE ACCURACY

results = {}

for filename in model_files:
    print("\nLoading:", filename)
    path = f"{MODELS_DIR}/{filename}"

    model = build_model_for_file(filename).to(device)
    state = torch.load(path, map_location=device)
    model.load_state_dict(state)

    acc = compute_train_accuracy(model, trainloader)
    results[filename] = acc
    print(f"  → Train Accuracy: {acc:.2f}%")

# SUMMARY

print("\n" + "="*60)
print("TRAINING ACCURACY SUMMARY")
print("="*60)
for name, acc in results.items():
    print(f"{name:<35} {acc:6.2f}%")
print("="*60)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found models:
  • MLP-Best_model.pth
  • CNN-Best-100ep_model.pth
  • CNN-Best-150ep_model.pth
  • MLP-Small-256_model.pth
  • MLP-Large-1024-512_model.pth
  • MLP-512-LR001_model.pth

Device: cuda

Loading: MLP-Best_model.pth
  → Train Accuracy: 60.53%

Loading: CNN-Best-100ep_model.pth
  → Train Accuracy: 94.19%

Loading: CNN-Best-150ep_model.pth
  → Train Accuracy: 95.21%

Loading: MLP-Small-256_model.pth
  → Train Accuracy: 50.26%

Loading: MLP-Large-1024-512_model.pth
  → Train Accuracy: 57.12%

Loading: MLP-512-LR001_model.pth
  → Train Accuracy: 51.79%

TRAINING ACCURACY SUMMARY
MLP-Best_model.pth                   60.53%
CNN-Best-100ep_model.pth             94.19%
CNN-Best-150ep_model.pth             95.21%
MLP-Small-256_model.pth              50.26%
MLP-Large-1024-512_model.pth         57.12%
MLP-512-LR001_model.pth              51.79%
